In [2]:
import pandas as pd
import random
import uuid
from datetime import datetime
import json
import time
import boto3  

In [3]:
# Cargar el dataset
df = pd.read_csv("fashionnova_reviews.csv")
print(df.shape)
df.head()


(131980, 9)


,Reviewer Name,Profile Link,Country,Review Count,Review Date,Rating,Review Title,Review Text,Date of Experience
0,Champagne,/users/66c78240087b6269ffbcb5fb,US,1 review,2024-08-22T20:24:02.000Z,Rated 5 out of 5 stars,I love ordering from fashion nova,I love ordering from fashion nova. The clothes...,"August 22, 2024"
1,Vg customer,/users/6618fdb53d4198001210cbe7,VG,3 reviews,2024-08-21T05:43:11.000Z,Rated 5 out of 5 stars,Top tier content for fashion nova,Always amazing clothes and the fast shipping i...,"August 18, 2024"
2,Colleen Burgher,/users/64e9595206be1a001244ff73,US,3 reviews,2024-08-21T17:09:14.000Z,Rated 5 out of 5 stars,Prices and quality of products are…,Prices and quality of products are GREAT Would...,"August 21, 2024"
3,R.G.M,/users/66c58ad1c6ab36352a08f57a,US,1 review,2024-08-21T08:36:03.000Z,Rated 5 out of 5 stars,Great customer service,Great customer service. I was helped until the...,"August 20, 2024"
4,Rosalyn Cousar,/users/60ad4b6ef3788e001adbb8e3,US,5 reviews,2024-08-22T00:46:16.000Z,Rated 3 out of 5 stars,False advertising,Disappointing experience. You don’t live up to...,"August 21, 2024"


In [4]:
# Convertir rating de texto a número entero
def procesar_rating(rating_str):
    try:
        return int(rating_str.split()[1])
    except:
        return None

df["rating_num"] = df["Rating"].apply(procesar_rating)

# Filtrar registros útiles
df = df[df["Review Text"].notna()]
df = df[df["rating_num"].notna()]

df.shape

(131980, 10)

In [5]:
# Canales simulados
canales = ["web", "movil", "call_center", "redes_sociales"]

# Función para limpiar valores no serializables
def safe_str(x):
    return str(x) if pd.notnull(x) else ""

def generar_mensaje():
    fila = df.sample(1).iloc[0]
    return {
        "id": str(uuid.uuid4()),
        "canal": random.choice(canales),
        "fecha": safe_str(fila["Review Date"]),
        "revisor": safe_str(fila["Reviewer Name"]),
        "pais": safe_str(fila["Country"]),
        "rating": int(fila["rating_num"]),
        "titulo": safe_str(fila["Review Title"]),
        "texto": safe_str(fila["Review Text"])
    }

In [6]:
lambda_client = boto3.client("lambda", region_name="us-east-1")

# Simular y mostrar mensajes
for _ in range(5):
    mensaje = generar_mensaje()
    
    # Solo se envía lo necesario a Lambda (texto y canal)
    payload = {
        "texto": mensaje["texto"],
        "canal": mensaje["canal"]
    }

    respuesta = lambda_client.invoke(
        FunctionName='AnalizarSentimiento',
        InvocationType='RequestResponse',
        Payload=json.dumps(payload).encode('utf-8')
    )

    print("Respuesta Lambda:", respuesta['Payload'].read().decode('utf-8'))
    time.sleep(1)

Respuesta Lambda: {"statusCode": 200, "body": "{\"sentimiento\": \"NEGATIVE\"}"}
Respuesta Lambda: {"statusCode": 200, "body": "{\"sentimiento\": \"POSITIVE\"}"}
Respuesta Lambda: {"statusCode": 200, "body": "{\"sentimiento\": \"POSITIVE\"}"}
Respuesta Lambda: {"statusCode": 200, "body": "{\"sentimiento\": \"NEGATIVE\"}"}
Respuesta Lambda: {"statusCode": 200, "body": "{\"sentimiento\": \"MIXED\"}"}
